In [ ]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import numpy as np
import xgboost
from rdkit import Chem
from rdkit.Chem import AllChem
import lazyqsar as lq
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt

DATAPATH = "../data"

In [ ]:
filename = os.path.join(DATAPATH,"Data.csv")
Data = pd.read_csv(filename)
Data

In [ ]:
SMILES = "SMILES"
ACTIVE = "Active"

In [ ]:
import matplotlib.pyplot as plt
x = Data[ACTIVE]
print(x)

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.countplot(x=x, data=Data)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')
plt.xlabel('Active')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()

activity_counts = Data['Active'].value_counts()
print(activity_counts)

In [ ]:
desc = Data['SMILES']
y = Data['Active']

In [ ]:
y = np.array(y)

#  Training with Train-test Split

# **Morgan BinaryClassifier_60**

In [ ]:
"""
  Generate your own train_test_split using the code below
  Uncomment to use
"""
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for i, (train, test) in enumerate(cv.split(desc, y)):
#     # Store training and testing sets for the current fold
#     train_set = pd.DataFrame(desc.iloc[train])
#     test_set = pd.DataFrame(desc.iloc[test])

#     # Add the Activity column to both training and testing sets
#     train_set['Active'] = y[train]
#     test_set['Active'] = y[test]

#     # Save the combined training and testing sets for the current fold
#     train_file = f"train_{i}.csv"
#     test_file = f"test_{i}.csv"
#     train_set.to_csv(train_file, index=False)
#     test_set.to_csv(test_file, index=False)

In [ ]:
# Define SMILES and EXP variables if not defined
SMILES = 'SMILES'
EXP = 'Active'

# Assuming 'desc' and 'y' are your input data
# Initialize AutoML for classification task
model = lq.MorganBinaryClassifier(time_budget_sec=60, estimator_list=["rf", "lgbm", "xgboost"])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_sets = []
test_sets = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
all_true_labels = []
all_predictions = []


for i in range(5):
    train_file = f"train_{i}.csv"
    test_file = f"test_{i}.csv"
    train_set = pd.read_csv(train_file)
    test_set = pd.read_csv(test_file)
    smiles_train = train_set['SMILES']
    print(smiles_train)
    print(y_train)
    y_train = train_set[EXP]

    # Append training and testing sets to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Fit the model on the training set for the current fold
    model.fit(smiles_train, y_train)

    # Obtain predictions and true labels for the current fold
    y_hat_proba = model.predict_proba(test_set[SMILES])[:, 1]
    fpr, tpr, thresholds = roc_curve(test_set[EXP], y_hat_proba)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

    y_pred = model.predict(test_set[SMILES])
    true_labels = test_set[EXP]

    # Append the training and testing sets for the current fold to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Classification report for the current fold
    print(f"\nClassification Report for Fold {i}:\n")
    print(classification_report(true_labels, y_pred))

    # Accumulate true labels and predictions
    all_true_labels.extend(test_set[EXP])
    all_predictions.extend(y_pred)

# Calculate and print the average classification report
print("\nAverage Classification Report Across All Folds:\n")
print(classification_report(all_true_labels, all_predictions))

# Calculate mean ROC curve and AUC
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Plot mean ROC curve with boundaries
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Cross-Validation ROC of AutoML', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# Confusion Matrix
model_name = 'MorganBinaryClassifier_60'
print(f"-------- {model_name} Confusion Matrix --------")
conf_mat =  confusion_matrix(y[test], y_pred)
ax = sns.heatmap(conf_mat, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

# **MorganBinaryClassifier_600**

In [ ]:
# Define SMILES and EXP variables if not defined
SMILES = 'SMILES'
EXP = 'Active'

# Assuming 'desc' and 'y' are your input data
# Initialize AutoML for classification task
model = lq.MorganBinaryClassifier(time_budget_sec=600, estimator_list=["rf", "lgbm", "xgboost"])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_sets = []
test_sets = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
all_true_labels = []
all_predictions = []


for i in range(5):
    train_file = f"train_{i}.csv"
    test_file = f"test_{i}.csv"
    train_set = pd.read_csv(train_file)
    test_set = pd.read_csv(test_file)
    smiles_train = train_set['SMILES']
    y_train = train_set[EXP]

    # Append training and testing sets to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Fit the model on the training set for the current fold
    model.fit(smiles_train, y_train)

    # Obtain predictions and true labels for the current fold
    y_hat_proba = model.predict_proba(test_set[SMILES])[:, 1]
    fpr, tpr, thresholds = roc_curve(test_set[EXP], y_hat_proba)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

    y_pred = model.predict(test_set[SMILES])
    true_labels = test_set[EXP]

    # Append the training and testing sets for the current fold to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Classification report for the current fold
    print(f"\nClassification Report for Fold {i}:\n")
    print(classification_report(true_labels, y_pred))

    # Accumulate true labels and predictions
    all_true_labels.extend(test_set[EXP])
    all_predictions.extend(y_pred)

# Calculate and print the average classification report
print("\nAverage Classification Report Across All Folds:\n")
print(classification_report(all_true_labels, all_predictions))

# Calculate mean ROC curve and AUC
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Plot mean ROC curve with boundaries
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Cross-Validation ROC of AutoML', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# Confusion Matrix
model_name = 'MorganBinaryClassifier_600'
print(f"-------- {model_name} Confusion Matrix --------")
conf_mat =  confusion_matrix(test_set[EXP], y_pred)
ax = sns.heatmap(conf_mat, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

# **ErsiliaBinaryClassifier_60**

In [ ]:
# Define SMILES and EXP variables if not defined
SMILES = 'SMILES'
EXP = 'Active'

# Assuming 'desc' and 'y' are your input data
# Initialize AutoML for classification task
model = lq.ErsiliaBinaryClassifier(time_budget_sec=60, estimator_list=["rf", "lgbm", "xgboost"])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_sets = []
test_sets = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
all_true_labels = []
all_predictions = []


#for i, (train, test) in enumerate(cv.split(desc, y)):
for i in range(5):
    train_file = f"train_{i}.csv"
    test_file = f"test_{i}.csv"
    train_set = pd.read_csv(train_file)
    test_set = pd.read_csv(test_file)
    smiles_train = train_set['SMILES']
    y_train = train_set[EXP]

    # Append training and testing sets to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Fit the model on the training set for the current fold
    model.fit(smiles_train, y_train)

    # Obtain predictions and true labels for the current fold
    y_hat_proba = model.predict_proba(test_set[SMILES])[:, 1]
    fpr, tpr, thresholds = roc_curve(test_set[EXP], y_hat_proba)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

    y_pred = model.predict(test_set[SMILES])
    true_labels = test_set[EXP]

    # Append the training and testing sets for the current fold to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Classification report for the current fold
    print(f"\nClassification Report for Fold {i}:\n")
    print(classification_report(true_labels, y_pred))

    # Accumulate true labels and predictions
    all_true_labels.extend(test_set[EXP])
    all_predictions.extend(y_pred)

# Calculate and print the average classification report
print("\nAverage Classification Report Across All Folds:\n")
print(classification_report(all_true_labels, all_predictions))

# Calculate mean ROC curve and AUC
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Plot mean ROC curve with boundaries
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Cross-Validation ROC of AutoML', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# Confusion Matrix
model_name = 'ErsiliaBinaryClassifier_60'
print(f"-------- {model_name} Confusion Matrix --------")
conf_mat =  confusion_matrix(test_set[EXP], y_pred)
ax = sns.heatmap(conf_mat, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

# **ErsiliaBinaryClassifier_600**

In [ ]:
# Define SMILES and EXP variables if not defined
SMILES = 'SMILES'
EXP = 'Active'

# Assuming 'desc' and 'y' are your input data
# Initialize AutoML for classification task
model = lq.ErsiliaBinaryClassifier(time_budget_sec=600, estimator_list=["rf", "lgbm", "xgboost"])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_sets = []
test_sets = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
all_true_labels = []
all_predictions = []


#for i, (train, test) in enumerate(cv.split(desc, y)):
for i in range(5):
    train_file = f"train_{i}.csv"
    test_file = f"test_{i}.csv"
    train_set = pd.read_csv(train_file)
    test_set = pd.read_csv(test_file)
    smiles_train = train_set['SMILES']
    y_train = train_set[EXP]

    # Append training and testing sets to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Fit the model on the training set for the current fold
    model.fit(smiles_train, y_train)

    # Obtain predictions and true labels for the current fold
    y_hat_proba = model.predict_proba(test_set[SMILES])[:, 1]
    fpr, tpr, thresholds = roc_curve(test_set[EXP], y_hat_proba)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

    y_pred = model.predict(test_set[SMILES])
    true_labels = test_set[EXP]

    # Append the training and testing sets for the current fold to the lists
    train_sets.append(train_set)
    test_sets.append(test_set)

    # Classification report for the current fold
    print(f"\nClassification Report for Fold {i}:\n")
    print(classification_report(true_labels, y_pred))

    # Accumulate true labels and predictions
    all_true_labels.extend(test_set[EXP])
    all_predictions.extend(y_pred)

# Calculate and print the average classification report
print("\nAverage Classification Report Across All Folds:\n")
print(classification_report(all_true_labels, all_predictions))

# Calculate mean ROC curve and AUC
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Plot mean ROC curve with boundaries
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Cross-Validation ROC of AutoML', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# Confusion Matrix
model_name = 'ErsiliaBinaryClassifier_600'
print(f"-------- {model_name} Confusion Matrix --------")
conf_mat =  confusion_matrix(test_set[EXP], y_pred)
ax = sns.heatmap(conf_mat, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

The model using ErsiliaBinaryClassifier and train time of 600 performed well than the other models. This is because as seen in the AUC-ROC curve. It has the highest mean AUC value (0.78) and the smallest standard deviation (0.08) for the 5-fold cross-validation. This indicates that the model has consistently performed well across all five folds of the cross-validation, suggesting better generalization ability compared to the other models.

The other three models have lower mean AUC values (0.75, 0.74, and 0.75) and larger standard deviations (0.10, 0.07, and 0.03), which indicates more variability in performance across the folds. This could be due to overfitting or other issues with the model.

In general, a lower standard deviation for the AUC ROC curve indicates greater consistency in model performance across different data splits. Therefore, the Mean ROC (AUC=0.78 +- 0.08) is the best model based on these metrics.

We would now proceed to training our entire data with this model

# **Training on the Entire data**

In [ ]:
# Assuming Data is your original dataframe
# Define SMILES and EXP variables if not defined
SMILES = 'SMILES'
EXP = 'Active'

no_of_fold = 5
results = []

for i in range(no_of_fold):
    # Use the entire dataset for training
    train = Data

    # Separate features (SMILES) and target variable (EXP) for training
    smiles_train = train[SMILES]
    y_train = train[EXP]

    # Initialize and fit the model: Our best model was
    model = lq.ErsiliaBinaryClassifier(time_budget_sec=600, estimator_list=["rf", "lgbm", "xgboost"])
    model.fit(smiles_train, y_train)

    # Separate features (SMILES) for testing (you can use the entire dataset or a different dataset for testing)
    smiles_test = Data[SMILES]

    # Predict probabilities using the trained model
    y_hat = model.predict_proba(smiles_test)

    # Store the results
    results.append(y_hat)

# Calculate the mean score
mean_score = np.mean(results, axis=0)
mean_score


In [ ]:
# We need the real results, the activity of the test set
y_test = Data['Active']

# We use the sklearn package to calculate the roc_curve and plot it
y_hat = y_hat[:,1]
fpr, tpr, _ = roc_curve(y_test, y_hat)
auroc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='#50285a', lw=2, label=f'ROC curve (AUC = {auroc:.2f})')
plt.plot([0, 1], [0, 1], color='#bee6b4', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
proba_cutoff = 0.5
y_hat_bin = [1 if x >= proba_cutoff else 0 for x in y_hat]


cf_matrix = confusion_matrix(y_test, y_hat_bin)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Greens', cbar=False, annot_kws={"size": 16})
ax.set_title("Predicted vs Real", fontsize=14)
ax.set_xlabel('Predicted', fontsize=14)
ax.set_ylabel('Real', fontsize=14)

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Inactive','Active'], fontsize=14)
ax.yaxis.set_ticklabels(['Inactive','Active'], fontsize=14)

In [ ]:
model.save("model_eosce_full600.joblib")